# Big Data Modelling and Management 2022

Group number: 24

Students (name : student number):


1. Ana Luís : 20210671

2. Carolina Machado : 20210676 

3. Francisco Calha : 20210673 

4. Sara Arana : 20210672


## 🚚 BDMM Second Homework Assignment 🚚 

_The Wide World Importers (WWI) is a wholesales novelty goods importer and distributor operating from the San Francisco bay area. In this assignment we will be working with their database._ 
You can get more information and details about the WWI database in the following link: https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-what-is?view=sql-server-ver15

The focus of the second assignment is modelling. We will use the World Wide Importers database and convert it to a document-based database. To that end, we will be leveraging concepts like data denormalization, indices, and mongodb design patterns. 

More information on the extended datamodel to be found here: </br>  
https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-oltp-database-catalog?view=sql-server-ver15

## Problem Description

Your team has just arrived at WWI (a leading company in logistics). Welcome!   <br>
Even though business is thriving, the IT department is going through a bad time.   <br>
Digitalization was never a priority for the company and now the company operational and analytical requirements are starting to grow beyond the capabilities of their existing data architecture.   <br>

WWI data are spread accross different systems, but we've already managed to pull them all into a mongo dump file. This data file is an exact dump of the SQL data so includes all the same structure, the SQL tables become collections and the rows become documents. This means all the original SQL keys are included in the data.<br>
Currently, the costs to develop the necessary queries to collect data to answer questions asked by the different departments are too high. <br>

Management concluded it is the right time to revise and revamp the data architecture, in order to speed up operations. 

In that context, your team was tasked with merging all the company data into a single and coherent Mongo database. <br>
It is expected that, with your solution, WWI will have a better understanding of their business and that the different departments will be able to obtain efficiently the answers they need.

The WWI team shared with you an ERD of their current datamodel:<br>
![datamodel](./WWI.png)

**Note** You can open the file WWI.png that is in the same directory as this notebook to see the above image in more detail and zoom in as you need.

Addtionally, the WWI team asked you the deliver the following outputs in **4 weeks**:
- Understand and model the database in MongoDB.
- Setup the database so that it is performs well for the queries they have provided. You should include reasoning in comments for the decisions you make on modelling the database.
- Answer the questions (queries) on the data provided.  
- Submit the results by following the instructions.  

With these deliveries, you will have created a prototype and allows the management to decide whether MongoDB is a good solution that meets their requirements.

### Design Requirements

Note that WWI has the following query requirements for the database.

1. The web team needs to know:  
    1. Which state province do we have the most suppliers in?  
    2. How many people have three or more `OtherLanguage`? 
    3. Top 10 most common `OtherLanguage` for people records. 
    4. How many customer records are valid after `November 2015`? 
    5. What percentage of people records don't have the UserPreferences field? 

2. The warehouse group needs to know:  
    1. What is the average difference in days between OrderDate and ExpectedDeliveryDate for orders sold by (`SalespersonPersonID`) person with the name `Jack Potter`?
    2. Which items get ordered the most in bulk (largest average quantity ordered)?  
    3. Which two items get ordered together the most?
    4. For each customer category which 3 items have the ordered the most?
    5. What is the current stock of each stockgroup?

3. The CFO needs to know:  
    1. What is the monthly total order count for each month?  
    2. How many orders are there from the customer `Tailspin Toys (Head Office)`?
    3. What are the average monthly sales prices of all goods sold? 
    4. In each state province what is the average customer credit limit?   
    5. What are the yearly expenditures with each supplier (per supplier name)?  

4. Partnerships needs to know:  
    1. What is the most common payment type?  
    2. What percentage of people have their `Title` as `Team Member`?
    3. Which supplier of the category `Novelty Goods Supplier` has the most transactions?  
    4. What is the highest `CommissionRate` that a person has?

5. The marketing team needs to know:  
    1. What is the name of the sales person with the largest sum of invoice values in 2013 (person whose customers paid the most money)?
    2. Who are the most common `PickedByPersonID` person names for orders done by customer `Adriana Pena`?
    3. How many people have in their name the string `Sara`?
    5. What are the top 10 most Common Names (Primary or Surnames) of people?

Transform the mongo dump file provided with this notebook and model a database following mongodb's best practices. You should adjust the data model to best fit the use cases provided above. Think about collections, embedding, linking, indexing, and the patterns learned in class. Provide justifications for each decision you make. What, if any, are the trade-off's or disadvantages of your approach.

Use MongoDB queries to answer the questions on your transformed database.

### Deliverables

1. Notebook with all DB creation operations and CRUD operations to create the data model. **Important** you should include in comments justification for your decisions on modelling the data.;
2. Second notebook with all required queries and answers for the questions, **Important** please indicate with comments the steps in the data model you took to optimise each query;


# Additional Information

## Groups  

Students should form groups of at least 4 and at most 5. <br>

## Submission  Deadline

The submission includes two notebooks with outputs (cells must be run). 
Please make sure to indicate:
1. group number,
2. group members with student names and numbers,
3. the name of the database that you created. <br>

Upload the notebook on moodle before **23:59 on June 22nd**

## Evaluation   

The second homework assignment counts 40% towards your final mark of the curricular unit. <br>
The assignment will be scored from 0 to 20. <br>

Each group submission will be evaluated on three components:
1. correctness of results;
2. simplicity and performance characteristics of the solution;
3. justification of decisions.

50% -  Database design  
50% -  Query results including performance

Please note that all code delivered in this assignment will go through plagiarism automated checks. <br>
Groups with high similarity levels in their code will undergo investigation.


### 6 rules of thumb of design process

1. Prefer embedding unless there is a compelling reason not to.
2. Needing to access an object on its own is a compelling reason not to embed it.
3. Arrays should not grow without bound. If there are more than a couple of hundred documents on the “many” side, don’t embed them; if there are more than a few thousand documents on the “many” side, don’t use an array of ObjectID references. High-cardinality arrays are a compelling reason not to embed.
4. Don’t be afraid of application-level joins: application-level joins are barely more expensive than server-side joins in a relational database.
5. Consider the write/read ratio when denormalizing. A field that will mostly be read and only seldom updated is a good candidate for denormalization: if you denormalize a field that is updated frequently then the extra work of finding and updating all the instances is likely to overwhelm the savings that you get from denormalizing.
6. As always with MongoDB, how you model your data depends – entirely – on your particular application’s data access patterns.

In [1]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint
from pymongo import TEXT
from datetime import datetime

host="localhost"
port="27017"
user="AzureDiamond"
password="hunter2"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")
db = client.WideWorldImporters

### 1. The web team needs to know:  

 A. Which state province do we have the most suppliers in? 

In [2]:
query_1 = {
    "$group": {
        "_id":{"Province Name": "$PostalStateProvince.StateProvinceName"},
        "Number of Suppliers": {"$sum": 1}
    }
}

query_2 = {
     "$sort": {"Number of Suppliers": -1}
 }

#Limit = 1, so we get the State Province with the biggest number of suppliers
query_3 = {'$limit': 1}

pipeline = [query_1, query_2, query_3]

r = db.supplier_location_embed.aggregate(pipeline)

result = list(r)

pprint(result)

[{'Number of Suppliers': 3, '_id': {'Province Name': ['California']}}]


B. How many people have three or more `OtherLanguage`?

In [3]:
query_1 = {'$unwind': '$OtherLanguages'}

query_2 = {
    "$group": {
        "_id": "$PersonID",
        "CountLanguages": {"$sum": 1}
    }
}

query_3 = {
    "$match": {
        "CountLanguages": {'$gte': 3}
    }
}

pipeline = [query_1, query_2, query_3]

r = db.referenced_people.aggregate(pipeline)

result = len(list(r))

print('The number of people that have three or more "OtherLanguages" is: ' + str(result) + '.')

The number of people that have three or more "OtherLanguages" is: 4.


 C. Top 10 most common `OtherLanguage` for people records. 

In [4]:
query_1 = {'$unwind': '$OtherLanguages'}

query_2 = {
    "$group": {
        "_id": "$OtherLanguages",
        "Count": {"$sum": 1}
    }
}

query_3 = {
    "$sort": {"Count": -1}
}

#Limit = 10 so we get the top 10 most common languages
query_4 = {
    "$limit": 10
}

pipeline = [query_1, query_2, query_3, query_4]

r = db.referenced_people.aggregate(pipeline)

result = list(r)

pprint(result)

[{'Count': 3, '_id': 'Greek'},
 {'Count': 3, '_id': 'Finnish'},
 {'Count': 3, '_id': 'Dutch'},
 {'Count': 2, '_id': 'Slovak'},
 {'Count': 2, '_id': 'Arabic'},
 {'Count': 2, '_id': 'Romanian'},
 {'Count': 2, '_id': 'Polish'},
 {'Count': 2, '_id': 'Croatian'},
 {'Count': 2, '_id': 'Lithuanian'},
 {'Count': 1, '_id': 'Slovenian'}]


 D. How many customer records are valid after `November 2015`? 

In [5]:
query_1 = {
    "$match": {"ValidTo" : {"$gt": datetime(2015,11,30)}}
}

pipeline = [query_1]

r = db.customers_embed.aggregate(pipeline)

result = len(list(r)) #gives the size of the list of the matched records

print('The number of customer records that are valid after November 2015 is: ' + str(result) + '.')

The number of customer records that are valid after November 2015 is: 663.


E. What percentage of people records don't have the UserPreferences field?

In [6]:
query_1 = {
    '$match':
        {'UserPreferences': None}}

pipeline = [query_1]

r = db.referenced_people.aggregate(pipeline)

result =  round(len(list(r))/(db.referenced_people.count_documents({}))*100,2) #compute the percentage

print("The percentage of people records that don't have the UserPreferences field is: " + str(result) + "%.")

The percentage of people records that don't have the UserPreferences field is: 83.62%.


### 2. The warehouse group needs to know:

A. What is the average difference in days between OrderDate and ExpectedDeliveryDate for orders sold by (SalespersonPersonID) person with the name Jack Potter?

In [7]:
#We performed this query before we did the lookup to optimize the query performance
#Will be faster because we have less records to join in the lookup
query_1 = {
    '$match': {'FullName': 'Jack Potter'}
        }

query_2 = {
        "$lookup":{
           "from": "referenced_orders",
           "localField": "PersonID",
           "foreignField": "SalespersonPersonID",
           "as": "Order"
        }
    }

query_3 = {"$unwind" : "$Order"}

#Group by '_id':0, so we can access all the records 
query_4 = {
         '$group':
             {   '_id': 0,
                 'averageTime':
                    {'$avg':
                          {'$dateDiff':
                                {   'startDate': '$Order.OrderDate',
                                    'endDate': '$Order.ExpectedDeliveryDate',
                                    'unit': "day"}
                           }
                       }
                   }
                }

query_5 = { '$project': { '_id': 0, 'Average Number of Days' : { '$trunc': [ "$averageTime", 1 ] } } }

pipeline = [query_1, query_2, query_3, query_4, query_5]

r = db.referenced_people.aggregate(pipeline)

result = list(r)

pprint(result)

[{'Average Number of Days': 1.4}]


B. Which items get ordered the most in bulk (largest average quantity ordered)?

In [8]:
#For this particular query, we only need the StockItemID (to join orderlines)
#and StockItemName (to print the result)
query_1 = {
    '$project': {
        'StockItemID': 1,
        'StockItemName': 1
    }
}

query_2 = {
        "$lookup":{
           "from": "referenced_orderlines",
           "localField": "StockItemID",
           "foreignField": "StockItemID",
           "as": "Order"
        }
    }

query_3 = {'$unwind': '$Order'}

query_4  = { "$group": {
             "_id": {'Stock Item Name': "$StockItemName"},
             "Average": { "$avg": "$Order.Quantity" }}}

query_5 = {'$sort': {"Average": -1}}
    
#Limit = 1 so we get the most ordered item in bulk
query_6 = {'$limit':1}
     
pipeline = [query_1, query_2, query_3, query_4, query_5, query_6]

r = db.stockitems_embed.aggregate(pipeline)

result = list(r)

pprint(result)

[{'Average': 199.35,
  '_id': {'Stock Item Name': 'Black and orange fragile despatch tape '
                             '48mmx75m'}}]


C. Which two items get ordered together the most?

In [9]:
#Sorting Description in an ascending order, so that the final results are always the same
#It would also worked in a descending order
#If we didn't do this step, the results would be always different, because of the order the items 
#would be pushed into the array
query_1 = {'$sort': {'Description':1}}


#We checked, and the Description field in orderlines matches the StockItemName in stockitems records
query_2 = {
    '$group': {
      '_id': {'OrderID': "$OrderID"},
      'Items': {'$push': "$Description"}
    }
  }

#For each array Items, creates arrays with only two items without repetition
#Each array is sorted in an ascending order by name, that was a consequence of query_1
#If we missed that step, cases in which arrays are like [item1, item2] , [item2, item1] would be counted as different
query_3 = {'$project': {
            'PairsOfItems': {"$reduce":
                      {'input':{'$range':[0,{'$size':"$Items"}]}, 
                       'initialValue':[], 
                       'in':{'$concatArrays':[ 
                             "$$value", 
                             {'$let':{
                             'vars':{'i':"$$this"},
                             'in':{'$map':{
                             'input':{'$range':[{'$add':[1,"$$i"]},{'$size':"$Items"}]},
                             'in':[ {'$arrayElemAt':["$Items","$$i"]}, {'$arrayElemAt':["$Items","$$this"]}] }}
                                  }}
                            ]}
                    } } 
            } }  


query_4 = {'$unwind' : '$PairsOfItems'}

query_5 = {'$group':{
            '_id':"$PairsOfItems",
            "Count": {"$sum": 1}
           }
        }

query_6 = {'$sort': {"Count": -1}}

#Limit = 2 because we have a draw    
query_7 = {'$limit':2}  


pipeline = [query_1,query_2,query_3,query_4,query_5,query_6,query_7]

r = db.referenced_orderlines.aggregate(pipeline)

result = list(r)

pprint(result)

[{'Count': 30,
  '_id': ['Animal with big feet slippers (Brown) XL',
          'Dinosaur battery-powered slippers (Green) S']},
 {'Count': 30,
  '_id': ['Air cushion film 200mmx200mm 325m',
          'Developer joke mug - fun was unexpected at this time (Black)']}]


D. For each customer category which 3 items have the ordered the most?

In [10]:
query_1 = {
        "$lookup":{
           "from": "referenced_orderlines",
           "localField": "OrderID",
           "foreignField": "OrderID",
           "as": "Orderline"
        }
    }

query_2 = {
        "$lookup":{
           "from": "customers_embed",
           "localField": "CustomerID",
           "foreignField": "CustomerID",
           "as": "Customer"
        }
    }

query_3 = {'$unwind': '$Orderline'}

query_4 = {'$unwind': '$Customer'}

query_5 = {'$unwind': '$Customer.CustomerCategory'}

query_6  = { "$group": {
        "_id": {'CustomerCategoryName':'$Customer.CustomerCategory.CustomerCategoryName', 
                'Item': '$Orderline.Description'},
        "Count" : {'$sum' : 1}
         }
  }


query_7 = {'$sort': {'Count': -1}}

query_8 = { "$group": {
            "_id": {'CustomerCategoryName': "$_id.CustomerCategoryName"},
            "TopItems": { "$push": "$_id.Item" }
          }
        }
   
#Slicing the array to get the 3 top items
#We can do this because of query_7, since the items were pushed to the array by count in a descending order
#If we didn't use this, the result wouldn't be accurate
query_9 = { "$project": { '_id':1,"Top3Items": { "$slice": ["$TopItems", 3] } } }

pipeline = [query_1, query_2, query_3, query_4, query_5, query_6, query_7, query_8, query_9]

r = db.referenced_orders.aggregate(pipeline)

result = list(r)

pprint(result)   

[{'Top3Items': ['Dinosaur battery-powered slippers (Green) L',
                'Halloween skull mask (Gray) L',
                'USB food flash drive - sushi roll'],
  '_id': {'CustomerCategoryName': 'Novelty Shop'}},
 {'Top3Items': ['"The Gu" red shirt XML tag t-shirt (Black) 6XL',
                '"The Gu" red shirt XML tag t-shirt (White) 7XL',
                'Superhero action jacket (Blue) XXS'],
  '_id': {'CustomerCategoryName': 'Supermarket'}},
 {'Top3Items': ['Alien officer hoodie (Black) 3XL',
                'Ride on toy sedan car (Pink) 1/12 scale',
                'Black and yellow heavy despatch tape 48mmx100m'],
  '_id': {'CustomerCategoryName': 'Computer Store'}},
 {'Top3Items': ['Red and white urgent despatch tape 48mmx75m',
                '"The Gu" red shirt XML tag t-shirt (Black) 6XL',
                'Alien officer hoodie (Black) 3XL'],
  '_id': {'CustomerCategoryName': 'Corporate'}},
 {'Top3Items': ['"The Gu" red shirt XML tag t-shirt (White) 3XS',
               

E. What is the current stock of each stockgroup?

In [11]:
query_1 = {'$unwind': '$StockGroup'}


query_2  = { "$group": {
        "_id": {'StockGroupName':'$StockGroup.StockGroupName'},
        "CurrentStock" : {'$sum' : '$QuantityPerOuter'}
         }
  }

    
pipeline = [query_1, query_2] 

r = db.stockitems_embed.aggregate(pipeline)

result = list(r)

pprint(result)

[{'CurrentStock': 360, '_id': {'StockGroupName': 'Novelty Items'}},
 {'CurrentStock': 42, '_id': {'StockGroupName': 'Mugs'}},
 {'CurrentStock': 312, '_id': {'StockGroupName': 'T-Shirts'}},
 {'CurrentStock': 492, '_id': {'StockGroupName': 'Clothing'}},
 {'CurrentStock': 14, '_id': {'StockGroupName': 'USB Novelties'}},
 {'CurrentStock': 21, '_id': {'StockGroupName': 'Toys'}},
 {'CurrentStock': 378, '_id': {'StockGroupName': 'Computing Novelties'}},
 {'CurrentStock': 68, '_id': {'StockGroupName': 'Furry Footwear'}},
 {'CurrentStock': 1010, '_id': {'StockGroupName': 'Packaging Materials'}}]


### 3. The CFO needs to know:

A. What is the monthly total order count for each month?

In [ ]:
#This question can have two interpretations, either we get the monthly total 
#order count for each month by year, or we get it for each month without considering the year

#We decided to execute the two approaches, since we thought that, in each case, the results would 
#provide relevant information

In [12]:
## By Month and Year

query_1 = { '$group': 
               { '_id': 
                { 
                  'Year': { '$year': "$OrderDate" }, 
                  'Month': { '$month': "$OrderDate" } }, 
                  'Count': { '$sum': 1 }
               }
          }

query_2 = {"$sort":{"_id":1}}


pipeline = [query_1, query_2] 

r = db.referenced_orders.aggregate(pipeline)

result = list(r)

pprint(result)

[{'Count': 1674, '_id': {'Month': 1, 'Year': 2013}},
 {'Count': 1139, '_id': {'Month': 2, 'Year': 2013}},
 {'Count': 1683, '_id': {'Month': 3, 'Year': 2013}},
 {'Count': 1696, '_id': {'Month': 4, 'Year': 2013}},
 {'Count': 1808, '_id': {'Month': 5, 'Year': 2013}},
 {'Count': 1675, '_id': {'Month': 6, 'Year': 2013}},
 {'Count': 1886, '_id': {'Month': 7, 'Year': 2013}},
 {'Count': 1537, '_id': {'Month': 8, 'Year': 2013}},
 {'Count': 1617, '_id': {'Month': 9, 'Year': 2013}},
 {'Count': 1618, '_id': {'Month': 10, 'Year': 2013}},
 {'Count': 1552, '_id': {'Month': 11, 'Year': 2013}},
 {'Count': 1565, '_id': {'Month': 12, 'Year': 2013}},
 {'Count': 1791, '_id': {'Month': 1, 'Year': 2014}},
 {'Count': 1538, '_id': {'Month': 2, 'Year': 2014}},
 {'Count': 1586, '_id': {'Month': 3, 'Year': 2014}},
 {'Count': 1739, '_id': {'Month': 4, 'Year': 2014}},
 {'Count': 1908, '_id': {'Month': 5, 'Year': 2014}},
 {'Count': 1887, '_id': {'Month': 6, 'Year': 2014}},
 {'Count': 2008, '_id': {'Month': 7, 'Year'

In [13]:
## By Month, in general

query_1 = {
    "$group": {
        "_id":  { '$month': "$OrderDate" },
        "Count": {"$sum": 1}
    }
}


query_2 = {
    "$sort": {"_id": 1}
}


pipeline = [query_1, query_2] 

r = db.referenced_orders.aggregate(pipeline)

result = list(r)

pprint(result)

[{'Count': 7239, '_id': 1},
 {'Count': 6115, '_id': 2},
 {'Count': 7129, '_id': 3},
 {'Count': 7497, '_id': 4},
 {'Count': 7722, '_id': 5},
 {'Count': 5551, '_id': 6},
 {'Count': 6167, '_id': 7},
 {'Count': 4908, '_id': 8},
 {'Count': 5319, '_id': 9},
 {'Count': 5504, '_id': 10},
 {'Count': 5014, '_id': 11},
 {'Count': 5430, '_id': 12}]


B. How many orders are there from the customer Tailspin Toys (Head Office)?

In [14]:
#We performed query_1 before we did the lookup to optimize the query performance
#Will be faster because we have less records to join in the lookup

query_1 = {
    "$match": {
        "CustomerName": 'Tailspin Toys (Head Office)' }
}


query_2 = {
        "$lookup":{
           "from": "referenced_orders",
           "localField": "CustomerID",
           "foreignField": "CustomerID",
           "as": "Order"
        }
    }

query_3 = {'$unwind': '$Order'}

query_4 = {
    "$group": {
        "_id":  "$Order.OrderID",
        "Count": {"$sum": 1}
    }
}

pipeline = [query_1, query_2, query_3,query_4] 

r = db.customers_embed.aggregate(pipeline)

result = len(list(r))

print("The number of orders that are from the customer Tailspin Toys (Head Office) is: " + str(result) + ".")

The number of orders that are from the customer Tailspin Toys (Head Office) is: 129.


C. What are the average monthly sales prices of all goods sold?

In [ ]:
#Similar to question 3A, here we can also have two interpretations

#For the same reasons we mentioned above, we will execute the two approaches

In [15]:
## By month, year

query_1 = {
        "$lookup":{
           "from": "referenced_orderlines",
           "localField": "OrderID",
           "foreignField": "OrderID",
           "as": "Orderline"
        }
    }

query_2 = {'$unwind' : '$Orderline'}

query_3 = { '$group':{
           '_id': 
                { 'Year': { '$year': "$OrderDate" }, 
                  'Month': { '$month': "$OrderDate" } 
               },
                  'AverageMonthlySales': { "$avg" : { "$multiply" : ["$Orderline.UnitPrice", 
                                                                       "$Orderline.Quantity"] } }
            }
          }

query_4 = {"$sort": {"_id": 1} }

query_5 = {'$project': { '_id':1,
                        'AverageMonthlySales': {'$trunc': ['$AverageMonthlySales',2]}
}}
pipeline = [query_1, query_2, query_3, query_4, query_5] 

r = db.referenced_orders.aggregate(pipeline)

result = list(r)

pprint(result)

[{'AverageMonthlySales': 724.26, '_id': {'Month': 1, 'Year': 2013}},
 {'AverageMonthlySales': 757.18, '_id': {'Month': 2, 'Year': 2013}},
 {'AverageMonthlySales': 735.95, '_id': {'Month': 3, 'Year': 2013}},
 {'AverageMonthlySales': 782.03, '_id': {'Month': 4, 'Year': 2013}},
 {'AverageMonthlySales': 800.63, '_id': {'Month': 5, 'Year': 2013}},
 {'AverageMonthlySales': 777.46, '_id': {'Month': 6, 'Year': 2013}},
 {'AverageMonthlySales': 763.69, '_id': {'Month': 7, 'Year': 2013}},
 {'AverageMonthlySales': 747.29, '_id': {'Month': 8, 'Year': 2013}},
 {'AverageMonthlySales': 767.99, '_id': {'Month': 9, 'Year': 2013}},
 {'AverageMonthlySales': 750.31, '_id': {'Month': 10, 'Year': 2013}},
 {'AverageMonthlySales': 770.58, '_id': {'Month': 11, 'Year': 2013}},
 {'AverageMonthlySales': 750.72, '_id': {'Month': 12, 'Year': 2013}},
 {'AverageMonthlySales': 739.63, '_id': {'Month': 1, 'Year': 2014}},
 {'AverageMonthlySales': 739.69, '_id': {'Month': 2, 'Year': 2014}},
 {'AverageMonthlySales': 772.81

In [16]:
## By month in general

query_1 = {
        "$lookup":{
           "from": "referenced_orderlines",
           "localField": "OrderID",
           "foreignField": "OrderID",
           "as": "Orderline"
        }
    }

query_2 = {'$unwind' : '$Orderline'}

query_3 = { '$group':{
           '_id': 
                {  
                  'Month': { '$month': "$OrderDate" } 
               },
                  'AverageMonthlySales': { "$avg" : { "$multiply" : ["$Orderline.UnitPrice", 
                                                                       "$Orderline.Quantity"] } }
            }
          }

query_4 = {"$sort": {"_id": 1} }

query_5 = {'$project': { '_id':1,
                        'AverageMonthlySales': {'$trunc': ['$AverageMonthlySales',2]}
}}

pipeline = [query_1, query_2, query_3, query_4, query_5] 

r = db.referenced_orders.aggregate(pipeline)

result = list(r)

pprint(result)

[{'AverageMonthlySales': 757.01, '_id': {'Month': 1}},
 {'AverageMonthlySales': 767.66, '_id': {'Month': 2}},
 {'AverageMonthlySales': 768.27, '_id': {'Month': 3}},
 {'AverageMonthlySales': 780.34, '_id': {'Month': 4}},
 {'AverageMonthlySales': 786.07, '_id': {'Month': 5}},
 {'AverageMonthlySales': 761.85, '_id': {'Month': 6}},
 {'AverageMonthlySales': 765.96, '_id': {'Month': 7}},
 {'AverageMonthlySales': 772.72, '_id': {'Month': 8}},
 {'AverageMonthlySales': 766.6, '_id': {'Month': 9}},
 {'AverageMonthlySales': 751.38, '_id': {'Month': 10}},
 {'AverageMonthlySales': 771.95, '_id': {'Month': 11}},
 {'AverageMonthlySales': 753.65, '_id': {'Month': 12}}]


D. In each state province what is the average customer credit limit?

In [17]:
query_1 = {
    '$match': {'CreditLimit': {'$exists' :1, '$ne' : None}
          }
}
    
query_2 = {
                "$group": {
                "_id":  {"StateProvinceName":"$PostalStateProvince.StateProvinceName"},
                "AverageCreditLimit": {"$avg": '$CreditLimit'}
            }
        }

query_3 = { '$project': { '_id': 1, 
                         'AverageCreditLimit' : { '$trunc': [ "$AverageCreditLimit", 2 ] } } } 

pipeline = [query_1, query_2, query_3]

r = db.customer_location_embed.aggregate(pipeline)

result = list(r)

pprint(result)

[{'AverageCreditLimit': 2900.0, '_id': {'StateProvinceName': ['Kansas']}},
 {'AverageCreditLimit': 2858.33, '_id': {'StateProvinceName': ['Alaska']}},
 {'AverageCreditLimit': 2773.33, '_id': {'StateProvinceName': ['Michigan']}},
 {'AverageCreditLimit': 1666.66, '_id': {'StateProvinceName': ['Montana']}},
 {'AverageCreditLimit': 1750.0, '_id': {'StateProvinceName': ['Maryland']}},
 {'AverageCreditLimit': 2950.0, '_id': {'StateProvinceName': ['Arizona']}},
 {'AverageCreditLimit': 2800.0,
  '_id': {'StateProvinceName': ['New Hampshire']}},
 {'AverageCreditLimit': 2481.66, '_id': {'StateProvinceName': ['Iowa']}},
 {'AverageCreditLimit': 2033.33, '_id': {'StateProvinceName': ['Colorado']}},
 {'AverageCreditLimit': 2915.0, '_id': {'StateProvinceName': ['Virginia']}},
 {'AverageCreditLimit': 2942.85, '_id': {'StateProvinceName': ['Ohio']}},
 {'AverageCreditLimit': 2595.0, '_id': {'StateProvinceName': ['Nebraska']}},
 {'AverageCreditLimit': 2758.92, '_id': {'StateProvinceName': ['New York']}},

E. What are the yearly expenditures with each supplier (per supplier name)?

In [18]:
#Regarding the transaction types, related to suppliers, we only have records
#with the transaction type '5' and '7', 'Supplier Invoice' and 'Supplier Payment Issued',
#respectively. The first one refers to the value of the transaction and the
#second one refers to the amount that the company has to pay to the supplier.
#For the same transaction, the absolute value of both is supposed to be the same 
#(when this value differs, from type '5' to type '7', it means that the invoice has emitted
#but the it was not paid yet).
#So, we only will use the 'Supplier Invoice' since its values are positive.
#in addition, a payment is only issued after the invoice has been emitted.

query_1 = {'$unwind': '$TransactionType'}

query_2 = {'$match': {'TransactionType.TransactionTypeName':'Supplier Payment Issued'}}

query_3 = {
        "$lookup":{
           "from": "suppliers_embed",
           "localField": "SupplierID",
           "foreignField": "SupplierID",
           "as": "Suppliers"
        }
    }

query_4 = {'$unwind': '$Suppliers'}

query_5 = { '$group':{
           '_id': 
                {'Year': { '$year': "$TransactionDate" },
                  'SupplierName' : '$Suppliers.SupplierName'},
                  'ExpendituresWithSupplier': { "$sum" : {'$abs': '$TransactionAmount'} }
            }
          }

query_6 = {'$sort' : {'ExpendituresWithSupplier' : 1}}

pipeline = [query_1, query_2, query_3, query_4, query_5, query_6]

r = db.suppliertransactions_embed.aggregate(pipeline)

result = list(r)

pprint(result)

[{'ExpendituresWithSupplier': 360.53,
  '_id': {'SupplierName': 'Contoso, Ltd.', 'Year': 2013}},
 {'ExpendituresWithSupplier': 7462.45,
  '_id': {'SupplierName': 'Graphic Design Institute', 'Year': 2013}},
 {'ExpendituresWithSupplier': 27525.3,
  '_id': {'SupplierName': 'A Datum Corporation', 'Year': 2016}},
 {'ExpendituresWithSupplier': 58443.01,
  '_id': {'SupplierName': 'The Phone Company', 'Year': 2013}},
 {'ExpendituresWithSupplier': 90639.0,
  '_id': {'SupplierName': 'Northwind Electric Cars', 'Year': 2013}},
 {'ExpendituresWithSupplier': 9671552.35,
  '_id': {'SupplierName': 'Litware, Inc.', 'Year': 2013}},
 {'ExpendituresWithSupplier': 59995486.21,
  '_id': {'SupplierName': 'Fabrikam, Inc.', 'Year': 2013}},
 {'ExpendituresWithSupplier': 65775421.019999996,
  '_id': {'SupplierName': 'Litware, Inc.', 'Year': 2014}},
 {'ExpendituresWithSupplier': 85238633.46,
  '_id': {'SupplierName': 'Litware, Inc.', 'Year': 2016}},
 {'ExpendituresWithSupplier': 144497536.52,
  '_id': {'SupplierN

### 4. Partnerships needs to know:

A. What is the most common payment type?

In [ ]:
# In this query, we only counted the payment type for the customer transactions, since we thought
# that the main focus of the Parternships would be regarding its consumers (customers), and not regarding
# the most common payment type of the suppleirs transactions.

In [31]:
query_1 = {'$unwind': '$PaymentMethod'}

query_2 = {
         '$group': {
          '_id': '$PaymentMethod.PaymentMethodName',
         'CountTransactions':{'$sum': 1}
      }
   }

query_3 = {'$sort': {"CountTransactions": -1}}
    
query_4 = {'$limit': 1}


pipeline = [query_1, query_2, query_3, query_4]

r = db.customertransactions_embed.aggregate(pipeline)

result = list(r)

pprint(result)

[{'CountTransactions': 26637, '_id': 'EFT'}]


B. What percentage of people have their Title as Team Member?


In [ ]:
# We created a TEXT index for the field CustomFields, so it could support
# searching for string content. In this case, the content "Team Member"

In [20]:
db.referenced_people.create_index(
    [('CustomFields', TEXT)],
    default_language='english',
    name = 'CustomFields_text'
)

query_1 = { '$match': { '$text': { '$search': "Team Member" } } }


pipeline = [query_1] 

r = db.referenced_people.aggregate(pipeline)

result = round(len(list(r))/db.referenced_people.count_documents({})*100,2)

print("The percentage of people records that have their title as Team Member is: " + str(result) + "%.")

The percentage of people records that have their title as Team Member is: 1.17%.


In [21]:
# drop index after using (we won't need it again)
db.referenced_people.drop_index('CustomFields_text')

C. Which supplier of the category Novelty Goods Supplier has the most transactions?

In [22]:
query_1 = {'$unwind': '$SupplierCategory'}

#We performed query_2 before we did the lookup to optimize the query performance
#Will be faster because we have less records to join in the lookup
query_2 = {'$match': {'SupplierCategory.SupplierCategoryName': 'Novelty Goods Supplier'}}

query_3 = {
        "$lookup":{
           "from": "suppliertransactions_embed",
           "localField": "SupplierID",
           "foreignField": "SupplierID",
           "as": "SupplierTransactions"
        }
    }

query_4 =  {
         '$project': {
          '_id': 0,
         'SupplierName': 1,
         'CountTransactions': {'$size': "$SupplierTransactions" }
      }
   }

query_5 = {'$sort': {"CountTransactions": -1}}
    
query_6 = {'$limit': 1}


pipeline = [query_1, query_2, query_3,query_4,query_5,query_6]

r = db.suppliers_embed.aggregate(pipeline)

result = list(r)

pprint(result)


[{'CountTransactions': 16, 'SupplierName': 'Graphic Design Institute'}]


D. What is the highest CommissionRate that a person has?

In [23]:
query_1 = { '$group' : { '_id': 0, 'max': { '$max' : "$CommissionRate" }}}

pipeline = [query_1]

r = db.referenced_people.aggregate(pipeline)

result = list(r)

pprint(result)

[{'_id': 0, 'max': 4.55}]


### 5. The marketing team needs to know:

A. What is the name of the sales person with the largest sum of invoice values in 2013 (person whose customers paid the most money)?

In [24]:
query_1 =  { '$set': {'InvoiceYear': {'$year': '$InvoiceDate'}}}


#We performed query_2 before we did the lookup in query_3 to optimize the query performance
#Will be faster because we have less records to join in the lookup
query_2 = {'$match' : {'InvoiceYear':2013}}

query_3 = {
        "$lookup":{
           "from": "referenced_invoicelines",
           "localField": "InvoiceID",
           "foreignField": "InvoiceID",
           "as": "Invoiceline"
        }
    }

query_4 = {"$unwind" : "$Invoiceline"}


#We performed this group before we did the lookup in query_6 to optimize the query performance
#Will be faster because we have less records to join in the lookup
query_5 = { '$group' : { 
                        '_id': {'SalespersonPersonID': '$SalespersonPersonID'}, 
                        'SumOfInvoiceValues': {'$sum': '$Invoiceline.ExtendedPrice'}}
          }
           
query_6 = {"$lookup":{
           "from": "referenced_people",
           "localField": "_id.SalespersonPersonID",
           "foreignField": "PersonID",
           "as": "SalesPerson"
        }
    }

query_7 = {'$unwind': '$SalesPerson'}

query_8 = {'$project': {'_id':0,
                       'SalesPersonName':'$SalesPerson.FullName',
                       'SumOfInvoiceValues':1}}

query_9 = {'$sort': {"SumOfInvoiceValues": -1}}
    
query_10 = {'$limit': 1}

pipeline = [query_1, query_2, query_3, query_4, query_5, query_6, query_7, query_8, query_9, query_10]

r = db.referenced_invoices.aggregate(pipeline)

result = list(r)

pprint(result)

[{'SalesPersonName': 'Hudson Onslow', 'SumOfInvoiceValues': 5593922.31}]


B. Who are the most common PickedByPersonID person names for orders done by customer Adriana Pena?

In [25]:
#We performed query_1 before we did the lookups to optimize the query performance
#Will be faster because we have less records to join in both of the lookups
query_1 = {
    '$match': {'CustomerName': 'Adriana Pena'}
}


query_2 = {
        "$lookup":{
           "from": "referenced_orders",
           "localField": "CustomerID",
           "foreignField": "CustomerID",
           "as": "Order"
        }
    }

query_3 = {"$unwind" : "$Order"}


query_4 = {
        "$lookup":{
           "from": "referenced_people",
           "localField": "Order.PickedByPersonID",
           "foreignField": "PersonID",
           "as": "PickedByPerson"
        }
    }


query_5 = {"$unwind" : "$PickedByPerson"}


query_6 = {
        "$group":{
            "_id":{
                "PickedByPersonName": "$PickedByPerson.FullName"
            }, 
            "Count":{ "$sum": 1 }
    }}

query_7 = {'$sort': {"Count": -1}}
    
query_8 = {'$limit': 3} #The limit is 3 because the we have a draw

pipeline = [query_1,query_2,query_3,query_4,query_5,query_6,query_7,query_8]

r = db.customers_embed.aggregate(pipeline)

result = list(r)

pprint(result)

[{'Count': 3, '_id': {'PickedByPersonName': 'Katie Darwin'}},
 {'Count': 3, '_id': {'PickedByPersonName': 'Piper Koch'}},
 {'Count': 3, '_id': {'PickedByPersonName': 'Anthony Grosse'}}]


C. How many people have in their name the string Sara?

In [ ]:
# We created a TEXT index for the field FullName, so it could support
# searching for string content. In this case, the content "Sara"

In [26]:
db.referenced_people.create_index(
    [('FullName', TEXT)],
    default_language='english',
    name = 'FullName_text'
)

query_1 = { '$match': { '$text': { '$search': "Sara" } } }


pipeline = [query_1] 

r = db.referenced_people.aggregate(pipeline)

result = len(list(r))

print("The number of people that have in their name the string Sara is: " + str(result) + ".")

The number of people that have in their name the string Sara is: 4.


In [27]:
# drop index after using (we won't need it again)
db.referenced_people.drop_index('FullName_text')

D. What are the top 10 most Common Names (Primary or Surnames) of people?

In [28]:
query_1 = { '$project' : 
           {'NamesList' : { '$split': ["$FullName", " "]}
            } 
          }

query_2 = { '$addFields': {
                'firstName': { '$slice': [ '$NamesList', 1 ] },
                'lastName': { '$slice': [ '$NamesList', -1 ] }
    } }

query_3 = { '$project': { 'Name': { '$concatArrays': [ "$firstName", "$lastName" ] } } }

query_4 = {
    "$unwind": "$Name"
}

query_5 = {'$group': {
            '_id': '$Name',
             "Count": { "$sum": 1 }
            } }

query_6 = {'$sort': {"Count": -1}}
    
query_7 = {'$limit':10}

pipeline = [query_1, query_2, query_3, query_4, query_5, query_6, query_7]

r = db.referenced_people.aggregate(pipeline)

result = list(r)

pprint(result)

[{'Count': 8, '_id': 'Bose'},
 {'Count': 7, '_id': 'Ganguly'},
 {'Count': 6, '_id': 'Thakur'},
 {'Count': 6, '_id': 'Roman'},
 {'Count': 5, '_id': 'Mukherjee'},
 {'Count': 5, '_id': 'PrabhupÄ\x81da'},
 {'Count': 5, '_id': 'De'},
 {'Count': 5, '_id': 'Dhanishta'},
 {'Count': 5, '_id': 'David'},
 {'Count': 4, '_id': 'Sara'}]
